In [ ]:
with open("/Users/abhishek/.profile", "r") as env_file:
    for line in env_file:
        if line.strip() != "":
            print(f"Line Data: '{line.strip()}'")
            key, value = line.strip().split(sep="=")
            os.environ[key] = value.replace('"','')

In [ ]:
from argparse import Namespace
from enum import Enum, auto
from logging import debug
import os
from typing import Dict, List
import yaml
from ccloud.core import create_ccloud_request, execute_ccloud_request, get_http_connection, generate_iso8601_dt_intervals
from ccloud.model import CCMEReq_Granularity
from data_processing.metrics_processing import metrics_dataframe
import helpers
from config_parser import try_parse_config_file, locate_storage_path, DirType
import datetime
import pandas as pd
import csv


core_config = try_parse_config_file(config_yaml_path="/Users/abhishek/Non-Cloud/repo/ccloud-chargeback-helper/config/config.yaml")

In [ ]:
storage_path = locate_storage_path(dir_type=[DirType.MetricsData, DirType.BillingsData, DirType.OutputData])

In [ ]:
intervals = generate_iso8601_dt_intervals(CCMEReq_Granularity.P1D.name, intervals=3)
print(intervals)

In [ ]:
new_req = create_ccloud_request(request=core_config["configs"]["connection"][0]["requests"][0], intervals=3)
print(new_req)

In [ ]:
connection = get_http_connection(ccloud_details=core_config["configs"]["connection"][0]["ccloud_details"])


In [ ]:
resp_code, resp_body = execute_ccloud_request(
    ccloud_url="https://api.telemetry.confluent.cloud/v2/metrics/cloud/query", auth=connection, payload=new_req, timeout=200
)
print(resp_body)

In [ ]:
curr_df = metrics_dataframe(aggregation_metric_name=new_req["aggregations"][0]["metric"], metrics_output=resp_body)

In [ ]:
curr_df.print_sample_df()

In [ ]:
# this is for the timestamp based pivoted dataset
from csv import QUOTE_NONNUMERIC

trial_out_basepath = storage_path[DirType.MetricsData]
# dt_range = curr_df.get_dataset(ds_name="pivoted_on_timestamp")[1].index.unique().normalize().unique()
dt_range = curr_df.get_dataset(ds_name="pivoted_on_timestamp")[1].index.unique().normalize().unique()
dates_arr = dt_range.date
print(dt_range)
print(dates_arr)

for dt_data, d_data in zip(dt_range, dates_arr):
    print(d_data)
    print(dt_data)
    new_item = dt_data + datetime.timedelta(days=1)
    print(new_item)
    print(type(new_item))
    ds = curr_df.get_dataset(ds_name="pivoted_on_timestamp")[1]
    subset = ds[(ds.index >= dt_data) & (ds.index < new_item)]
    out_path = os.path.join(trial_out_basepath, f"{d_data}_pivoted_on_timestamp.csv")
    subset.to_csv(out_path, quoting=QUOTE_NONNUMERIC)
    

In [ ]:
ds = curr_df.get_dataset(ds_name="metricsapi_representation")[1]
ts_range = ds["timestamp"].dt.normalize().unique()
print(ts_range)
dt_range = ts_range.date
print(dt_range)
print(type(ts_range))
print(type(dt_range))


In [ ]:
import csv
for ts_date, dt_str in zip(ts_range, dt_range):
    lt_date = ts_date + datetime.timedelta(days=1)
    print(ts_date, dt_str, lt_date)
    subset = ds[(ds["timestamp"] >= ts_date) & (ds["timestamp"] < lt_date)]
    print(subset.info())
    out_path = os.path.join(trial_out_basepath, f"{dt_str}_metricsapi_representation.csv")
    subset.to_csv(out_path, index=False, quoting=csv.QUOTE_NONNUMERIC)


In [ ]:
pip install pandas

In [ ]:
import pandas as pd

test1 = [
    {"test": 1,"idx": "index1", "value1": "test1value1","value2": "test1value2"},
    {"test": 2,"idx": "index2", "value1": "test2value1","value2": "test2value2"},
    {"test": 3,"idx": "index3", "value1": "test3value1","value2": "test3value2"},
    {"test": 4,"idx": "index4", "value1": "test4value1","value2": "test4value2"}
]

test2 = [
    {"test": 1,"idx": "index5", "value1": "test1value1","value2": "test1value2"},
    {"test": 2,"idx": "index6", "value1": "test2value1","value2": "test2value2"},
    {"test": 3,"idx": "index7", "value1": "test3value1","value2": "test3value2"},
    {"test": 4,"idx": "index8", "value1": "test4value1","value2": "test4value2"}
]


test3 = pd.concat([pd.DataFrame.from_records(test1, index=["test", "idx"]), pd.DataFrame.from_records(test2, index=["test", "idx"])])

print(test3)

test4 = test3[ (test3["test"] > 2) ]
print(test4)


In [ ]:
import datetime

test_date = datetime.datetime.now()
str_date = str(test_date.date())

print(str_date)

In [ ]:
import pandas

def generate_date_range_per_row(
    freq: str = "1H", override_start_date: str = None, override_end_date: str = None
):
    start_date = datetime.datetime.fromisoformat(override_start_date)
    end_date = datetime.datetime.fromisoformat(override_end_date)
    end_date = end_date - datetime.timedelta(minutes=1)
    return pd.date_range(start_date, end_date, freq=freq)


start_date = datetime.datetime.now().date() - datetime.timedelta(days=10)
last_date = datetime.datetime.now().date() - datetime.timedelta(days=1)

print(f"{start_date} -- {last_date}")

ts_range = generate_date_range_per_row(override_start_date=str(start_date), override_end_date=str(last_date))
# print(ts_range)

coll = [
    {"ts": item, "idx": index, "value1": f"value{index}"} for index, item in enumerate(ts_range)
]
# pprint.pprint(coll)

ds = pandas.DataFrame.from_records(coll)
# print(ds.info())

filter_date = last_date - datetime.timedelta(days=5)
filter_start_date = pd.to_datetime(str(filter_date))
filter_date = last_date - datetime.timedelta(days=4)
filter_end_date = pd.to_datetime(str(filter_date))
subset = ds.loc[(ds["ts"] >= filter_start_date) & (ds["ts"] < filter_end_date)]
subset.head()


In [ ]:
class MetricsAPIPrometheusName:
    request_bytes_name = "request_bytes_value"
    response_bytes_name = "response_bytes_value"
    request_bytes_value = "sum by (kafka_id, principal_id) (confluent_kafka_server_request_bytes)"
    response_bytes_value = "sum by (kafka_id, principal_id) (confluent_kafka_server_response_bytes)"

    def override_column_names(self, key, value):
        object.__setattr__(self, key, value)

METRICS_API_PROMETHEUS_NAME = MetricsAPIPrometheusName()

test_key = METRICS_API_PROMETHEUS_NAME.request_bytes_name
print(test_key)

print(METRICS_API_PROMETHEUS_NAME.__getattribute__(test_key))

In [ ]:
int1, int2 = 0, None
str1, str2 = "", None
list1, list2, list3 = [], ["test"], None
dict1, dict2, dict3 = {}, {"key": "my_value"}, None

val = [int1, int2, str1, str2, list1, list2, list3, dict1, dict2, dict3]
for item in val:
    print(f"{item} is None - {item is None} \t\t\t == None {item == None} \t\t\t not check {not item}")